<a href="https://colab.research.google.com/github/slin35/RobotProducer/blob/main/robotproducer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

In [12]:
# import data
data1 = pd.read_csv('./tmdb_5000_movies.csv')
data2 = pd.read_csv('./tmdb_5000_credits.csv')

data1 = data1[['genres', 'title', 'overview', 'id']]
data2 = data2[['movie_id', 'title', 'cast', 'crew']]

data = pd.merge(data1, data2, left_on='id', right_on='movie_id', how='inner')
data = data[['title_x', 'overview', 'genres', 'cast', 'crew', 'id']]
data.rename({'title_x': 'title'}, axis=1, inplace=True)



In [13]:
# cleanup genres
data['genres'] = [json.loads(i) if i != [] else [] for i in data['genres']]
data['genres'] = [[j['name'] if 'name' in j else [] for j in i] for i in data['genres']]

# cleanup cast
data['cast'] = [json.loads(i) if i != [] else [] for i in data['cast']]
data['cast'] = [[j['name'] if 'name' in j else [] for j in i] for i in data['cast']]

# cleanup crew
data['crew'] = [json.loads(i) if i != [] else [] for i in data['crew']]
data['crew'] = [[j['name']  for j in i if 'job' in j and j['job'] == 'Director'] for i in data['crew']]


In [4]:
# tokenize overview and remove stopwords and punctuation
'''
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))

data['overview'] = [word_tokenize(line) if type(line) is str else [] for line in data['overview']]

data['overview'] = [[w for w in line if w not in stop_words and w not in string.punctuation] for line in data['overview']]

# lemmatize words

lemmatizer = WordNetLemmatizer()
data['overview'] = [[lemmatizer.lemmatize(w) for w in line] for line in data['overview']]
'''

"\nnltk.download('stopwords')\nnltk.download('punkt')\nnltk.download('wordnet')\n\nstop_words = set(stopwords.words('english'))\n\ndata['overview'] = [word_tokenize(line) if type(line) is str else [] for line in data['overview']]\n\ndata['overview'] = [[w for w in line if w not in stop_words and w not in string.punctuation] for line in data['overview']]\n\n# lemmatize words\n\nlemmatizer = WordNetLemmatizer()\ndata['overview'] = [[lemmatizer.lemmatize(w) for w in line] for line in data['overview']]\n"

In [5]:
!pip install bert-tensorflow==1.0.1

In [6]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub

print(tf.__version__)


TensorFlow 1.x selected.
1.15.2


In [14]:
import bert

from bert import run_classifier
from bert import optimization
from bert import tokenization
#from bert import bert_tokenization
from bert import modeling
from bert import run_classifier_with_tfhub

In [15]:
# BERT-based, uncased model: 12-layer, 768-hidden, 12-heads, 110M parematers

BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"


def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])    
  return tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)


tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [16]:
# convert data to BERT format

# getting distinct genres
genres = []
[[genres.append(item) for item in row if item not in genres] for row in data['genres']]

# add in columns for each genre
for genre in genres:
  data[genre] = 0

# setup new columns to correct values
for i, row in data.iterrows():
  for genre in row['genres']:
    data.at[i, genre] = 1


# splitting data into training and test set
train, test = train_test_split(data, test_size = 0.5)

'''
# create BERT's InputExample from data
train_inputExamples = []
test_inputExamples = []

for i, row in train.iterrows():
  example = bert.run_classifier.InputExample(guid=None,
                                             text_a = row['overview'],
                                             text_b = None,
                                             label = row[5:])
  train_inputExamples.append(example)
  
for i, row in test.iterrows():
  example = bert.run_classifier.InputExample(guid=None,
                                             text_a = row['overview'],
                                             text_b = None,
                                             label = row[5:])
  test_inputExamples.append(example)


MAX_SEQ_LENGTH = 128
label_list = [0, 1]

# convert train and test features to BERT's InputFeatures 
train_features = bert.run_classifier.convert_examples_to_features(train_inputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_inputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
'''

# InputExample for object with multiple labels
class InputExample(object):
  def __init__(self, guid, text_a, text_b=None, labels=None):
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.labels = labels


class InputFeatures(object):
  def __init__(self, input_ids, input_mask, segment_ids, label_ids, is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_ids = label_ids
    self.is_real_example = is_real_example


In [17]:
train_inputExamples = []
test_inputExamples = []

for i, row in train.iterrows():
  example = InputExample(row['id'], row['overview'], row[5:])
  train_inputExamples.append(example)

for i, row in test.iterrows():
  example = InputExample(row['id'], row['overview'], row[5:])
  test_inputExamples.append(example)

In [28]:
# for a single sequence convert_example_to_features
def convert_examples_to_features(examples, max_seq_length, tokenizer):
  features = []

  for (idx, example) in enumerate(examples):
    if type(example.text_a) is not str:
      continue
    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None;

    if len(tokens_a) > max_seq_length - 2:
      tokens_a = tokens_a[:(max_seq_length - 2)]

    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_ids = [0] * len(tokens)           # its a single sequence, so segment_ids are all 0

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    input_mask = [1] * len(input_ids)

    # padding up to sequence length
    padding = [0] * (max_seq_length - len(input_ids))
    input_ids += padding
    input_mask += padding 
    segment_ids += padding 

    labels_ids = []
    if example.labels:
      for label in example.labels:
        labels_ids.append(int(label))

    feature = InputFeatures(input_ids, input_mask, segment_ids, labels_ids)
    features.append(feature)

  return features

MAX_SEQ_LENGTH = 128

train_features = convert_examples_to_features(train_inputExamples, MAX_SEQ_LENGTH, tokenizer)
test_features = convert_examples_to_features(test_inputExamples, MAX_SEQ_LENGTH, tokenizer)

In [19]:

# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [20]:
num_train_steps = int(len(train_inputExamples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

run_config = tf.estimator.RunConfig(
    model_dir = './Output',
    save_summary_steps = SAVE_SUMMARY_STEPS,
    save_checkpoints_steps = SAVE_CHECKPOINTS_STEPS
)

In [21]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [22]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [23]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': './Output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faf793fbc10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': './Output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faf793fbc10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


AttributeError: ignored